In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [9]:
!apt-get install openjdk-8-jdk-headless -qq > /dev/null
!wget -q https://www-us.apache.org/dist/spark/spark-2.4.7/spark-2.4.7-bin-hadoop2.7.tgz
!tar xf /content/spark-2.4.7-bin-hadoop2.7.tgz
!pip install -q findspark

E: Failed to fetch http://security.ubuntu.com/ubuntu/pool/universe/o/openjdk-8/openjdk-8-jre-headless_8u282-b08-0ubuntu1~18.04_amd64.deb  404  Not Found [IP: 91.189.88.142 80]
E: Failed to fetch http://security.ubuntu.com/ubuntu/pool/universe/o/openjdk-8/openjdk-8-jdk-headless_8u282-b08-0ubuntu1~18.04_amd64.deb  404  Not Found [IP: 91.189.88.142 80]
E: Unable to fetch some archives, maybe run apt-get update or try with --fix-missing?


In [5]:
import os
os.environ["JAVA_HOME"] = "/usr/lib/jvm/java-8-openjdk-amd64"
os.environ["SPARK_HOME"] = "/content/spark-2.4.7-bin-hadoop2.7"

In [6]:
import findspark
findspark.init()

In [7]:
from pyspark.sql import SparkSession
spark = SparkSession.builder\
        .appName("Spark ML_Regression Application")\
        .master('local[*]')\
        .getOrCreate()

Exception: ignored

In [ ]:
from pyspark.sql.types import *
from pyspark.sql.functions import *

In [ ]:
train_sparkdf = spark.read.csv(path='/content/drive/MyDrive/Final_Hack/train_without_Outlier.csv',header=True,inferSchema=True)


In [ ]:
train_df=train_sparkdf
df1=train_df.drop('_c0','onpromotion','city','class','year','item_id','locationId','dayofweek','date','type')

In [ ]:
df1.show(1)

+----------+-------------+-------+--------------------+----------+-----+---+
|unit_sales|        state|cluster|    category_of_item|perishable|month|day|
+----------+-------------+-------+--------------------+----------+-----+---+
|       7.0|Uttar Pradesh|      1|baked_items / bre...|         1|    1|  1|
+----------+-------------+-------+--------------------+----------+-----+---+
only showing top 1 row



In [ ]:
df2=df1.limit(int(limnum))

In [ ]:
limnum=(df1.count()/2)+0.5

In [ ]:
df2.count()

45956723

In [ ]:
(trainingData, testData) = df2.randomSplit([0.7, 0.3], seed= 123)

In [ ]:
num_Var_Names = [c[0] for c in trainingData.dtypes if c[1] in ['double']]
cat_Var_Names = [c[0] for c in trainingData.dtypes if c[1] in ['string','int']]
print(num_Var_Names)
print(cat_Var_Names)

['unit_sales']
['state', 'cluster', 'category_of_item', 'perishable', 'month', 'day']


In [ ]:
num_Var_Names.remove('unit_sales')

In [ ]:
from pyspark.ml.feature import VectorAssembler

assembler_Num = VectorAssembler(inputCols=num_Var_Names, outputCol="num_features")

In [ ]:
from pyspark.ml.feature import OneHotEncoder, StringIndexer, VectorAssembler

indexers_Cat = [StringIndexer(inputCol=cat_Var_Name, outputCol="{0}_index".format(cat_Var_Name)) for cat_Var_Name in cat_Var_Names ]
encoders_Cat = [OneHotEncoder(inputCol=indexer.getOutputCol(), outputCol="{0}_vec".format(indexer.getInputCol())) for indexer in indexers_Cat]
assembler_Cat = VectorAssembler(inputCols=[encoder.getOutputCol() for encoder in encoders_Cat], outputCol="cat_features")

assembler = VectorAssembler(inputCols=["cat_features"], outputCol="features")

In [ ]:
preprocessiong_Stages = indexers_Cat+encoders_Cat+[assembler_Cat]+[assembler]

In [ ]:
from pyspark.ml.regression import LinearRegression

lr = LinearRegression(maxIter=100,labelCol="unit_sales", featuresCol="features")

In [ ]:
from pyspark.ml import Pipeline

lr_Pipeline = Pipeline(stages=preprocessiong_Stages+[lr])

lr_Pipeline_model = lr_Pipeline.fit(trainingData)

**Decision Tree Regressor**

---



In [ ]:
from pyspark.ml.regression import DecisionTreeRegressor

dt = DecisionTreeRegressor(labelCol="unit_sales", featuresCol="features")

In [ ]:
from pyspark.ml import Pipeline
dt_Pipeline = Pipeline(stages=preprocessiong_Stages+[dt]) 

dt_Pipeline_model = dt_Pipeline.fit(trainingData)

In [ ]:
train_predictions_dt = dt_Pipeline_model.transform(trainingData)
test_predictions_dt = dt_Pipeline_model.transform(testData)

In [ ]:
test_predictions_dt.show(2)

+--------------------+-----------+-------+--------------------+----------+-----+---+-----------+-------------+----------------------+----------------+-----------+---------+--------------+--------------+--------------------+--------------+--------------+---------------+--------------------+--------------------+------------------+
|          unit_sales|      state|cluster|    category_of_item|perishable|month|day|state_index|cluster_index|category_of_item_index|perishable_index|month_index|day_index|     state_vec|   cluster_vec|category_of_item_vec|perishable_vec|     month_vec|        day_vec|        cat_features|            features|        prediction|
+--------------------+-----------+-------+--------------------+----------+-----+---+-----------+-------------+----------------------+----------------+-----------+---------+--------------+--------------+--------------------+--------------+--------------+---------------+--------------------+--------------------+------------------+
|      

In [ ]:
from pyspark.ml.evaluation import RegressionEvaluator

evaluator = RegressionEvaluator(labelCol="unit_sales",
                            predictionCol="prediction",
                            metricName="rmse" )


In [ ]:
dtTrain_rmse = evaluator.evaluate(train_predictions_dt)
print('RMSE value on Train data is', dtTrain_rmse)

RMSE value on Train data is 4.281342138789577


In [ ]:
test_sparkdf = spark.read.csv(path='/content/drive/MyDrive/Final_Hack/test.csv',header=True,inferSchema=True)
loc_sparkdf = spark.read.csv(path='/content/drive/MyDrive/Final_Hack/location_details.csv',header=True,inferSchema=True)
item_sparkdf = spark.read.csv(path='/content/drive/MyDrive/Final_Hack/item_details.csv',header=True,inferSchema=True)

In [ ]:
loc_sparkdf.printSchema()

root
 |-- location_id: string (nullable = true)
 |-- city: string (nullable = true)
 |-- state: string (nullable = true)
 |-- type: string (nullable = true)
 |-- cluster: integer (nullable = true)



In [ ]:
item_sparkdf.printSchema()

root
 |-- item_id: string (nullable = true)
 |-- category_of_item: string (nullable = true)
 |-- class: string (nullable = true)
 |-- perishable: integer (nullable = true)



In [ ]:
loc=loc_sparkdf.withColumnRenamed("location_id", "locationId")

In [ ]:
loc.show(1)

+----------+---------+---------+----+-------+
|locationId|     city|    state|type|cluster|
+----------+---------+---------+----+-------+
|location_1|Bengaluru|Karnataka|   D|     13|
+----------+---------+---------+----+-------+
only showing top 1 row



In [ ]:
sdf1=test_sparkdf.join(other=loc,on='locationId',how='left')

In [ ]:
sdf2=sdf1.join(other=item_sparkdf,on='item_id',how='left')

In [ ]:
sdf2.show(1)

+----------+-----------+---+-------------------+-----------+------+-------------+----+-------+----------------+----------+----------+
|   item_id| locationId| id|               date|onpromotion|  city|        state|type|cluster|category_of_item|     class|perishable|
+----------+-----------+---+-------------------+-----------+------+-------------+----+-------+----------------+----------+----------+
|item_99197|location_25|  0|2019-01-01 00:00:00|      false|Kanpur|Uttar Pradesh|   D|      1|   grocery_items|class_1067|         0|
+----------+-----------+---+-------------------+-----------+------+-------------+----+-------+----------------+----------+----------+
only showing top 1 row



In [ ]:
from pyspark.sql.functions import dayofmonth
from pyspark.sql.functions import to_date

In [ ]:
sdf3=sdf2.withColumn('day',dayofmonth(sdf2.date))

In [ ]:
sdf3.show(1)

+----------+-----------+---+-------------------+-----------+------+-------------+----+-------+----------------+----------+----------+---+
|   item_id| locationId| id|               date|onpromotion|  city|        state|type|cluster|category_of_item|     class|perishable|day|
+----------+-----------+---+-------------------+-----------+------+-------------+----+-------+----------------+----------+----------+---+
|item_99197|location_25|  0|2019-01-01 00:00:00|      false|Kanpur|Uttar Pradesh|   D|      1|   grocery_items|class_1067|         0|  1|
+----------+-----------+---+-------------------+-----------+------+-------------+----+-------+----------------+----------+----------+---+
only showing top 1 row



In [ ]:
sdf3=sdf3.withColumn('month',month(sdf2.date))

In [ ]:
sdf3.show(3)

+-----------+-----------+---+-------------------+-----------+------+-------------+----+-------+--------------------+----------+----------+---+-----+-----+
|    item_id| locationId| id|               date|onpromotion|  city|        state|type|cluster|    category_of_item|     class|perishable|day|date1|month|
+-----------+-----------+---+-------------------+-----------+------+-------------+----+-------+--------------------+----------+----------+---+-----+-----+
| item_99197|location_25|  0|2019-01-01 00:00:00|      false|Kanpur|Uttar Pradesh|   D|      1|       grocery_items|class_1067|         0|  1|    1|    1|
|item_103665|location_25|  1|2019-01-01 00:00:00|      false|Kanpur|Uttar Pradesh|   D|      1|baked_items / bre...|class_2712|         1|  1|    1|    1|
|item_105574|location_25|  2|2019-01-01 00:00:00|      false|Kanpur|Uttar Pradesh|   D|      1|       grocery_items|class_1045|         0|  1|    1|    1|
+-----------+-----------+---+-------------------+-----------+------+--

In [ ]:
sdf4=sdf3.drop('date1')

In [ ]:
sdf4.show(1)

+----------+-----------+---+-------------------+-----------+------+-------------+----+-------+----------------+----------+----------+---+-----+
|   item_id| locationId| id|               date|onpromotion|  city|        state|type|cluster|category_of_item|     class|perishable|day|month|
+----------+-----------+---+-------------------+-----------+------+-------------+----+-------+----------------+----------+----------+---+-----+
|item_99197|location_25|  0|2019-01-01 00:00:00|      false|Kanpur|Uttar Pradesh|   D|      1|   grocery_items|class_1067|         0|  1|    1|
+----------+-----------+---+-------------------+-----------+------+-------------+----+-------+----------------+----------+----------+---+-----+
only showing top 1 row



In [ ]:
sdf5=sdf4.drop('id','onpromotion','city','class','item_id','locationId','date','type')

In [ ]:
sdf5.show(1)

+-------+----------------+----------+---+-----+
|cluster|category_of_item|perishable|day|month|
+-------+----------------+----------+---+-----+
|      1|   grocery_items|         0|  1|    1|
+-------+----------------+----------+---+-----+
only showing top 1 row



In [ ]:
dtTest_rmse = evaluator.evaluate(test_predictions_dt)
print('RMSE value on Test data is', dtTest_rmse)

RMSE value on Test data is 4.283739078052711


In [ ]:
test_predictions_dt_og = dt_Pipeline_model.transform(sdf5)

In [ ]:
final=test_predictions_dt_og.select('prediction')

In [ ]:
final.show(1)

+-----------------+
|       prediction|
+-----------------+
|4.539246205978413|
+-----------------+
only showing top 1 row



In [ ]:
import pandas as pd

In [ ]:
ss=final.limit(8501)

In [ ]:
final.count()

23517680

In [ ]:
ff=final.withColumn('prediction',col('prediction').cast('int'))

In [ ]:
ss=ff.limit(8500)
ss.toPandas()

,prediction
0,4
1,4
2,4
3,4
4,4
...,...
8495,5
8496,5
8497,5
8498,5


In [ ]:

from pyspark.sql.functions import monotonically_increasing_id 

df_index = final.select("*").withColumn("id", monotonically_increasing_id())


In [ ]:
sp=df_index.filter(df_index.id>1)
sp.filter(sp.id<10).show()

Py4JJavaError: ignored

In [ ]:
i=0
j=1
while (i < final.count()):
  path='/content/drive/MyDrive/Final_Hack/result1.csv'
  
  df=df_index.where(df_index.id>=i & df_index.id<=((i+1)*8500))
  i=j*8500
  j=j+1
  df.write.format('csv').option('header',True).mode('append').option('sep',',').save(path)
  

Py4JJavaError: ignored

In [ ]:
final.count()

23517680

Py4JError: ignored